In [1]:
from utils import *
from models import *
from experiments import *
from qd_data import *
import keras
import numpy as np
from keras.applications.mobilenet_v2 import MobileNetV2
from imgaug import augmenters as iaa
import imgaug as ia
%matplotlib inline

Using TensorFlow backend.


### Load Data

In [2]:
# df_all = pd.read_csv(data_path+"df_all_raw.csv")


In [3]:
small_df = pd.read_csv("../data/small-set.csv")




In [4]:
small_df.shape

(1000, 3)

In [5]:
exp1.train(MobileNetV2(input_shape=(32, 32, 1), alpha=1., weights=None, classes=num_classes),
          small_df,small_df,continue_training=True)

Epoch 1/2
1/1 [==============================] - 6s 6s/step - loss: 6.0744 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00 - val_loss: 14.9166 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 73ms/step - loss: 6.1668 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00 - val_loss: 12.5090 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00


In [29]:
# _10fold0 = load_obj("../data/folds10.pik")[0]
# train_index = _10fold0["train_index"]
valid_index = _10fold0["test_index"]

In [31]:
df_all.iloc[valid_index].to_csv("../data/valid_df.csv",index=False)

In [5]:
small_df = pd.read_csv("../data/small-set.csv")

In [148]:
valid_df = small_df

In [6]:
# train_gen = QD_Datagen(df_all.drawing.iloc[train_index],df_all.target.iloc[train_index],4, is_train_mode = True)
train_gen = QD_Datagen(sample_df.drawing,
                       sample_df.target,
                       batch_size,
                       is_train_mode = True, imsize=imsize)


In [44]:
valid_gen = QD_Datagen(valid_df.drawing,
                       valid_df.target,
                       batch_size, 
                       is_train_mode=False,
                       imsize=imsize)

### Model

In [2]:
imsize=96	  
batch_size=128
# fold = 8 #np.random.choice( range(1,10))

name = "xception_real_3d"
exp_name = f"{name}_im{imsize}_batch{batch_size}"

In [3]:
# exp_name = "xception_augv2_im128_fold5_nonrecxxx"
# callbacks = get_callbacks(exp_name, batch_size=batch_size)


In [3]:
# model = keras.applications.InceptionResNetV2(input_shape=(imsize, imsize, 1), weights=None, classes=num_classes)
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy', top_3_accuracy])

In [6]:
model.load_weights(f"../weights/{exp_name}.hdf5")

In [11]:
hist = model.fit_generator(valid_gen,
                           steps_per_epoch=10,#len(valid_gen)//(batch_size*10000),
                           epochs = 2,
                           validation_data = valid_gen,
                           validation_steps=10,  
                           callbacks = callbacks,
                           verbose = 1)

Epoch 1/2
 9/10 [==========================>...] - ETA: 1s - loss: 6.3528 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0x7fc67e0bd7f0>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
10/10 [==============================] - 19s 2s/step - loss: 6.4495 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00 - val_loss: 5.8291 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00
Epoch 2/2
10/10 [==============================] - 1s 81ms/step - loss: 7.3167 - acc: 0.0000e+00 - top_3_accuracy: 0.0250 - val_loss: 5.8293 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00


### Test prediction

In [2]:
imsize=96	  
batch_size=128
# fold = 8 #np.random.choice( range(1,10))

name = "xception_real_3d"
exp_name = f"{name}_im{imsize}_batch{batch_size}"

In [3]:
# exp_name = "xception_augv2_im128_fold5_nonrecxxx"

In [4]:
exp = GZ_experiment(imsize, batch_size, exp_name,
 preprocess_input=keras.applications.xception.preprocess_input, 
 num_workers=os.cpu_count(),
 min_lr=2e-5) 

In [5]:
exp.predict()

Predicting using model found in /home/ubuntu/draw/weights/xception_real_3d_im96_batch128xxxxxxx.hdf5 ..
877/877 [==============================] - 145s 165ms/step


In [30]:
test_predictions = model.predict_generator(test_gen, steps=np.ceil(len(test_df)/batch_size), verbose=1)
test_predictions = test_predictions[:test_df.shape[0]]

877/877 [==============================] - 81s 92ms/step


In [7]:
exp = Experiment(imsize,batch_size,exp_name)

In [8]:
exp.predict(num_augmentations=5)

Predicting using model found in /home/ubuntu/draw/weights/xception_augv2_im128_fold5_nonrecxxx.hdf5 ..
877/877 [==============================] - 273s 311ms/step


In [17]:
exp_name

'xception_augv2_im128_fold5_nonrecxxx'

In [6]:
test_predictions = np.load(f"../predictions/{exp_name}.npy")

In [13]:
test_predictions = [np.load("../predictions/denseNet121_im128_fold8_nonrecx.npy"),
                    np.load("../predictions/xception_augv2_im128_fold5_nonrecxxx_augx5.npy"),
                    ]
                            

In [14]:
test_predictions = np.mean(test_predictions,axis=0)

In [14]:
test_df = pd.read_csv('/home/ubuntu/draw/data/test_simplified.csv')

In [31]:
classfiles

[]

In [32]:

data_path ="/home/ubuntu/draw/data/"
train_path = data_path+"train_simplified/"
num_classes = 340 
BASE_SIZE = 256
NCATS = 340 
NCSVS = 100

classfiles = glob(train_path+"*.csv")
nametoid = {v[:-4].split("/")[-1].replace(" ", "_") :i
							 for i, v in enumerate(classfiles) if "df_all_raw" not in v} #adds underscores
idtoname = {i: v[:-4].split("/")[-1].replace(" ", "_")
							 for i, v in enumerate(classfiles) if "df_all_raw" not in v} #adds underscores


In [55]:
def list_all_categories():
    files = os.listdir(train_path)
    return sorted([f2cat(f) for f in files], key=str.lower)
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

In [58]:
top3 = preds2catids(test_predictions)
top3.head()


,a,b,c
0,234,281,238
1,144,36,226
2,305,62,53
3,187,127,241
4,56,113,112


In [61]:
cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)

In [48]:
test_top3_id = preds2catids(test_predictions)
top3cats = test_top3_id.replace(idtoname)


In [49]:
top3cats.shape, test_df.shape

((112199, 3), (112199, 4))

In [63]:
top3cats.a.value_counts()

coffee_cup                 629
calendar                   526
cake                       521
tornado                    520
peas                       436
bus                        432
cell_phone                 432
teddy-bear                 429
pickup_truck               410
toe                        405
jacket                     404
golf_club                  401
ambulance                  398
shorts                     398
raccoon                    397
potato                     397
grapes                     396
brain                      392
marker                     392
face                       391
leaf                       389
The_Great_Wall_of_China    389
paintbrush                 385
bulldozer                  384
violin                     384
goatee                     382
belt                       382
barn                       381
tiger                      381
pencil                     381
                          ... 
keyboard                   274
piano   

In [64]:
test_df['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test_df[['key_id', 'word']]
submission.to_csv('/home/ubuntu/draw/submissions/{}.csv'.format(exp_name), index=False)

In [65]:
exp_name

'xception_real_3d_im96_batch128'